# Week 6 Tuesday

## Sentiment Analysis the FastAI way

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/defreez/cs356-notebooks/blob/main/notebooks/week6-tue.ipynb)


In [4]:
!pip install -Uqq fastbook
import fastbook
from fastbook import *
fastbook.setup_book()

## The text equivalent of our magic six lines for cats and dogs

- Load a pretrained classifer for text, in this case an LSTM
- Load some movie reviews
- Fine tune it

In [7]:
# This model was pre-trained on Wikipedia
# This takes about 12GB of GPU memory
dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.466419,0.394005,0.823360,00:48


epoch,train_loss,valid_loss,accuracy,time
0,0.304442,0.297238,0.879280,01:26
1,0.236180,0.205868,0.920200,01:26
2,0.183470,0.187277,0.929160,01:26
3,0.146075,0.188952,0.929840,01:25


# Sentiment Analysis

- Binary dependent variable (positive sentiment, negative sentiment)
- Independent variable


## Language Model

- Try to predict the next word of the sentence
- Pre-train the model on Wikipedia
- Fine tune the last layer to predict sentiment of movie reviews

# Custom language model for Movie Reviews

- A word is basically a categorical variable
    - The **vocabulary**
- Create a random embedding matrix
- Use the embedding matrix as the first layer of a neural network
- Train to produce embedding

## Constructing the embedding

- Initialize words already in pre-trained model to their weight
- New words get random vectors


## Tokenization

- Convert the text to a list of tokens using spacy
- Replace each token with its index in vocab dictionary
- Is `don't` one token or two?
- What about Japanese or Chinese? What's a word?

word-based: split a sentence on spaces, some rules for punctuation.
sub-word: split word int osmaller parts
character: split a sentence into characters

Exercise: create a character-based tokenizer.

FastAI uses external tokenizers.

In [208]:
imdb_path = untar_data(URLs.IMDB)
(imdb_path/'train').ls()

(#4) [Path('/root/.fastai/data/imdb/train/neg'),Path('/root/.fastai/data/imdb/train/labeledBow.feat'),Path('/root/.fastai/data/imdb/train/pos'),Path('/root/.fastai/data/imdb/train/unsupBow.feat')]

In [209]:
get_text_files??

In [8]:
files = get_text_files(imdb_path, folders = ['train', 'test', 'unsup'])

In [212]:
first_review = files[1].open().read()
first_review

'This is blatantly a futuristic adaptation of Jules Verne\'s "Mysterious Island". The sound editing is pretty bad. You hear the dialogue on set and you hear the voices being recorded on a recording booth at the same time! This is an amateur film with actors from Boston and shot around New Hampshire. For those living in New Engalnd and who is reading this comment will be wowed with a capital W. This film is full of flaws. You get to hear the director\'s voice giving directions and giving out directions to the actress. "OK now stand up." As for the other characters. There is this guy who talks with his mind instead of his voice and this blue alien. The alien guy talks with a deep voice. When he is yawning or grunting when he is fighting you hear the actor\'s voice. As for the special effects, man! This was Brett Piper\'s early work for crying out loud! The creatures are good but the animation is jerky. Really jerky. Sort of like Karl Zeman animation in JOURNEY TO BEGINNING OF TIME (1955)

In [213]:
spacy = WordTokenizer()
toks = spacy([first_review])
first(toks)

(#264) ['This','is','blatantly','a','futuristic','adaptation','of','Jules','Verne',"'s"...]

In [200]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

## Tokenizer vs. WordTokenizer

- WordTokenizer is a wrapper around spaCy
- Tokenizer is FastAI wrapper around WordTokenizer with even more stuff

In [51]:
tkn = Tokenizer(spacy)
toks = tkn(first_review)
toks

(#362) ['xxbos','xxmaj','no','wonder','this','movie','never','saw','the','light'...]

- xxbos is tart of text (beginning of stream)
- xxmaj from capitalization

See docs at https://fastai1.fast.ai/text.transform.html

In [14]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

## Numericalization

- Maps tokens to integers

In [78]:
first_200_reviews = L(o.open().read() for o in files[:200])
toks200 = first_200_reviews.map(tkn)

# toks200 is an L of lists
toks200

(#200) [['xxbos', 'xxmaj', 'no', 'wonder', 'this', 'movie', 'never', 'saw', 'the', 'light', 'of', 'day', '.', 'xxmaj', 'the', 'timing', 'was', 'of', 'the', 'release', 'was', 'awful', '.', 'xxmaj', 'the', 'xxmaj', 'gong', 'xxmaj', 'show', 'had', 'already', '"', 'jumped', 'the', 'shark', '"', 'by', 'the', 'time', 'the', 'movie', 'came', 'out', ',', 'so', 'who', 'would', 'pay', 'money', 'just', 'to', 'see', 'a', 'few', 'of', 'the', 'censored', 'clips', 'from', 'the', 'original', 'run', 'of', 'the', 'show', '?', 'xxmaj', 'and', 'the', 'show', 'clips', 'are', 'just', 'a', 'tiny', 'bit', 'of', 'this', 'pathetic', ',', '90', '-', 'minute', 'whine', 'by', 'xxmaj', 'chuck', 'xxmaj', 'barris', 'about', 'how', 'hard', 'his', 'life', 'was', 'as', 'host', 'of', 'the', 'show', '.', 'xxmaj', 'did', 'he', 'really', 'expect', 'we', 'would', 'feel', 'sorry', 'for', 'him', 'and', 'his', 'messed', '-', 'up', 'millionaire', 'life', '?', 'xxmaj', 'did', 'he', 'really', 'think', 'we', 'even', 'wanted', 'to',

In [79]:
num = Numericalize()
num.setup(toks200)
len(num.vocab)

2024

In [83]:
num.vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 'the',
 ',',
 '.',
 'a',
 'and',
 'of',
 'to',
 'is',
 'it',
 'i',
 'in',
 'this',
 'that',
 '"',
 '-',
 '\n\n',
 'was',
 'movie',
 "'s",
 'for',
 'but',
 'as',
 'film',
 'with',
 'on',
 'not',
 'you',
 "n't",
 'be',
 'are',
 ')',
 'have',
 '(',
 'they',
 'one',
 'he',
 'all',
 'so',
 'like',
 'or',
 'just',
 'his',
 'at',
 'who',
 'about',
 '!',
 'an',
 'by',
 '?',
 'from',
 'do',
 'there',
 'if',
 'what',
 'has',
 'even',
 'some',
 'would',
 "'",
 'no',
 'which',
 'bad',
 '…',
 'only',
 'when',
 'her',
 'more',
 'up',
 'can',
 'out',
 'were',
 'we',
 'could',
 'good',
 'my',
 'time',
 'me',
 'really',
 'she',
 'acting',
 'did',
 'than',
 'story',
 ':',
 'very',
 'had',
 'their',
 'made',
 'movies',
 'most',
 'then',
 'into',
 'much',
 'been',
 'get',
 'see',
 'any',
 'because',
 'does',
 'too',
 'plot',
 'them',
 'first',
 'people',
 'films',
 'how',
 'seen',
 'other',
 'ever',
 'its',
 'after'

In [84]:
nums200 = toks200.map(num)
nums200

(#200) [TensorText([   2,    8,   68,  706,   20,   26,  126,  179,    9,  951,   14,  345,   11,    8,    9, 1480,   25,   14,    9,  952,   25,  189,   11,    8,    9,    8,    0,    8,  153,   94,  388,   22,
        1167,    9,    0,   22,   56,    9,   84,    9,   26,  346,   78,   10,   46,   52,   66,  632,  180,   49,   15,  104,   12,  152,   14,    9,    0, 1168,   58,    9,  199,  526,   14,    9,
         153,   57,    8,   13,    9,  153, 1168,   38,   49,   12, 1169,  266,   14,   20,  953,   10,  633,   23,  527,    0,   56,    8, 1481,    8,    0,   53,  114,  295,   50,  141,   25,   30,
           0,   14,    9,  153,   11,    8,   89,   44,   86,  528,   80,   66,  307,  410,   28,  133,   13,   50,    0,   23,   76,    0,  141,   57,    8,   89,   44,   86,  123,   80,   64,  347,
          15,    7,  124,   53,   50,  141,   57,   41,  437,   46,   10,  308,   44,  245, 1170,   50, 1171,    0,   53,   50,  954,   30,   12,    7,    0,    0,   11,   39,    8,   89, 

### Batches

- FastAI language batch size is define a little differently that batches for SGD
- For the language loader the batch size is the number of batches to divide the dataset into
    - Number of rows in a batch is batch size
    - Sequence length is also a parameter

In [85]:
dl = LMDataLoader(nums200)

# This is the size of one batch
x, y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [86]:
# The number of batches
len(dl)

13

## The prediction task

- Given preceding sequence of N tokens (here 72)
- Predict the next token (current sequence shifted over by 1)

In [90]:
' '.join(num.vocab[o] for o in x[0])

'xxbos xxmaj no wonder this movie never saw the light of day . xxmaj the timing was of the release was awful . xxmaj the xxmaj xxunk xxmaj show had already " jumped the xxunk " by the time the movie came out , so who would pay money just to see a few of the xxunk clips from the original run of the show ? xxmaj and the show clips are'

In [91]:
' '.join(num.vocab[o] for o in y[0])

'xxmaj no wonder this movie never saw the light of day . xxmaj the timing was of the release was awful . xxmaj the xxmaj xxunk xxmaj show had already " jumped the xxunk " by the time the movie came out , so who would pay money just to see a few of the xxunk clips from the original run of the show ? xxmaj and the show clips are just'

### Training

- Use is_lm=True for language modeling task (predicting next token)

In [94]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(imdb_path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1),
).dataloaders(imdb_path, path=imdb_path, bs=128, seq_len=80)

In [96]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj mississippi xxmaj burning is one of my favorite xxmaj gene xxmaj hackman movies . i remember watching it when i was young , it was one of the few rated r movies my parents would allow me to watch . xxmaj such a powerful story about the disappearance of three civil rights workers down in rural xxmaj mississippi . xxmaj when the xxmaj feds are called in to investigate the disappearance , the action really heats up .","xxmaj mississippi xxmaj burning is one of my favorite xxmaj gene xxmaj hackman movies . i remember watching it when i was young , it was one of the few rated r movies my parents would allow me to watch . xxmaj such a powerful story about the disappearance of three civil rights workers down in rural xxmaj mississippi . xxmaj when the xxmaj feds are called in to investigate the disappearance , the action really heats up . xxmaj"
1,"world ( who had opportunity to watch it ) accept this movie , because you have to know some moments in our serbian history and character of xxmaj serbs in the first half of the 20th century , to be able to understand it ! xxmaj but as i see here , there is somebody from xxmaj canada who watched it … and he liked it . \n\n i think that xxmaj i 'll try to put all good quotes","( who had opportunity to watch it ) accept this movie , because you have to know some moments in our serbian history and character of xxmaj serbs in the first half of the 20th century , to be able to understand it ! xxmaj but as i see here , there is somebody from xxmaj canada who watched it … and he liked it . \n\n i think that xxmaj i 'll try to put all good quotes from"


In [97]:
# Tuning the model for IMDB

learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [98]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.016117,3.900004,0.300612,49.402653,12:00


In [101]:
learn.save('imdb-1epoch')

Path('/root/.fastai/data/imdb/models/imdb-1epoch.pth')

In [214]:
### Movie Review Generation

TEXT = "Thanksgiving"
N_WORDS = 100
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
preds

['Thanksgiving dinner for Thanksgiving was a fun and exciting experience . The " breakfast Club " ( though the " two " title " thanksgiving Dinner " was more ironic for the kids ) was a pre - holiday feel to " breakfast at Tiffany \'s " ( the film \'s specialty is an American Pie " to Eat " ) . The cast introduced a Refreshing twist to the film , and Paul Rudd , Jamie Foxx , and Jamie Lee Curtis are',
 'Thanksgiving is one of the most fun movies i have seen in a long time . The cast is superb , and the film is fun . The special effects are a large percentage of the time . The story is simple and simple . Ben Hogan ( a lookalike ) is Oscar winning actor . He is the son of a wealthy family . He is his family , his brother , Uncle Frank , and his business manager , John Ireland , played by John']

In [121]:
# Make entire network trainable
learn.unfreeze()

In [122]:
# Train everything with a lower learning rate.
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.758005,3.756276,0.317301,42.788780,12:49
1,3.692900,3.698847,0.323663,40.400700,12:22
2,3.626208,3.648373,0.329389,38.412132,12:01
3,3.578109,3.615339,0.333142,37.163944,11:54
4,3.497035,3.594973,0.335827,36.414730,11:55
5,3.428614,3.578593,0.338395,35.823086,11:57
6,3.354045,3.569620,0.339711,35.503094,11:57
7,3.295889,3.567371,0.340822,35.423328,12:02
8,3.246153,3.571206,0.341171,35.559448,12:01
9,3.209833,3.575130,0.340937,35.699245,12:00


In [123]:
learn.save_encoder('imdb-finetuned')

### Experiment Shakespeare generation

In [111]:
shakespeare_path = Path('data/shakespeare')

shakespeare_dls_lm = DataBlock(
    blocks=TextBlock.from_folder(shakespeare_path, is_lm=True),
    get_items=get_text_files, splitter=RandomSplitter(0.1),
).dataloaders(shakespeare_path, path=shakespeare_path, bs=128, seq_len=80)

shakespeare_learn = language_model_learner(
    shakespeare_dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [112]:
shakespeare_learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.938603,3.796566,0.314286,44.547943,01:06


In [216]:
TEXT = "I am sure thou"  
N_WORDS = 64
N_SENTENCES = 2
preds = [shakespeare_learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
preds

["i am sure thou canst , thou mightst have made me swear \n▁ That i had never been an honest man . \n▁ Hold , there is a man 's son , and all thy fortunes \n▁ Are all mine ; and thine is mine , \n▁ And never shall thy husband lie in 's grave , \n▁ Nor hold thee till i",
 'i am sure thou hast been ever so ill - fed , \n▁ And thou hast made no envy in me since . \n▁ Or thou shouldst have been thy mother , \n▁ When thou hast a wife , and with a mother \n▁ Have i not seen thy son so tender . \n▁ SON . For she hath a daughter ,']

In [ ]:
shakespeare_learn.unfreeze()
shakespeare_learn.fit_one_cycle(10, 2e-3)

In [139]:
learn.save_encoder('shakespeare-finetuned')

### Using our custom language model for sentiment analysis

- Now we have two blocks, not one
- Independent variable is text
- Dependent category is sentiment, positive or negative

In [161]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(imdb_path, vocab=dls_lm.vocab), CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(imdb_path, path=imdb_path, bs=128, seq_len=72)

In [162]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos xxmaj some have praised _ xxunk _ as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think _ the xxmaj dirty xxmaj dozen _ ) but _ atlantis _ represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before",neg


- Can't split into batches like we did before because we want to use an entire review for classification
- This is done by adding padding tokens to make thinsg equal length
- FastAI puts similar length documents in mini-batches
- This is handled automatically

In [178]:
imdb_sent_learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()

In [179]:
imdb_sent_learn = imdb_sent_learn.load_encoder('imdb-finetuned')

In [181]:
imdb_sent_learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.252063,0.179821,0.930320,00:23


In [226]:
imdb_sent_learn.predict("This was a waste of my time.")

('neg', tensor(0), tensor([0.9466, 0.0534]))

## Disinformation

- Can generate context-appropriate sentences
- Keynote at scipy 2019

https://www.youtube.com/watch?v=KChtdexd5Jo&list=PLtmWHNX-gukLQlMvtRJ19s7-8MrnRV6h6&index=6

In [ ]:
https://github.com/EleutherAI/the-pile